# Batch Training with Ray Datasets

### Learning objectives
In this this tutorial, you will learn about:
 * [Ray Dataset](#dataset)
 * [Batch training with Ray Dataset](#train_func)

Batch training and tuning are common tasks in simple machine learning use-cases such as time series forecasting. They require fitting of simple models on multiple data batches corresponding to locations, products, etc. 

Batch training in the context of this notebook is understood as creating the same model(s) for different and separate datasets or subsets of a dataset. This notebook showcases how to conduct batch training using [Ray Dataset](https://docs.ray.io/en/latest/data/dataset.html).

<img src="./images/embarrassingly_parallel.png" width="50%" />

For the data, we will use the [NYC Taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).  This popular tabular dataset contains historical taxi pickups by timestamp and location in NYC.  <s>The goal is to predict future, hourly taxi demand by location in NYC.</s>  To demonstrate batch training & tuning, we will simplify the data to a linear regression problem to predict `trip_duration` and use Scikit-learn.

To demonstrate how data and training can be batch-parallelized, we will train a separate model for each dropoff location. This means we can use the dropoff_location_id column in the dataset to group the dataset into data batches. Then we will fit a separate model for each batch. 

Let’s start by importing a few required libraries, including open-source [Ray](https://github.com/ray-project/ray) itself!

In [1]:
import os, time
import random
from typing import Tuple, List, Union, Optional, Callable
import pandas as pd
import numpy as np
import pyarrow.dataset as pds
from pyarrow import fs
from ray.data import Dataset
from ray.data.preprocessors import Chain, OrdinalEncoder, StandardScaler

num_available_cpus = os.cpu_count()
print(f'Number of CPUs in this system: {num_available_cpus}')

# import utility functions
import local_utils.dataprep

Number of CPUs in this system: 8


In [2]:
import ray
if ray.is_initialized():
    ray.shutdown()
ray.init(ignore_reinit_error=True)

2022-10-12 21:24:37,219	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.8.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8266


In [3]:
# For benchmarking purposes, we can print the times of various operations. 
# In order to reduce clutter in the output, this is set to False by default.
PRINT_TIMES = True

def print_time(msg: str):
    if PRINT_TIMES:
        print(msg)

In [4]:
# To speed things up, we’ll only use a small subset of the full dataset consisting of two last months of 2019. 
# You can choose to use the full dataset for 2018-2019 by setting the SMOKE_TEST variable to False.

SMOKE_TEST = True

# Ray Dataset <a class="anchor" id="dataset"></a>

Next, read some data using Ray Dataset.   This will initialize a Ray cluster.  Then we can use the [Ray Dataset](https://docs.ray.io/en/latest/data/getting-started.html#datasets-getting-started) APIs to quickly inspect the data.

In [5]:
# Define some global variables.
target = "trip_duration"
s3  = fs.S3FileSystem(region="us-east-2")
s3_partitions = pds.dataset("ursa-labs-taxi-data/", filesystem=s3, partitioning=["year", "month"])

if SMOKE_TEST:
    starting_idx = -1
    sample_locations = random.sample(list(local_utils.dataprep.location_ids), 3)
else:
    starting_idx = -3
    sample_locations = list(local_utils.dataprep.location_ids)

s3_files = [f"s3://{file}" for file in s3_partitions.files][starting_idx:]
print(f"NYC Taxi using {len(s3_files)} file(s)!")
print(f"s3_files: {s3_files}")
print(f"sample locations: {sample_locations}")


NYC Taxi using 1 file(s)!
s3_files: ['s3://ursa-labs-taxi-data/2019/06/data.parquet']
sample locations: [238, 157, 205]


In [6]:
# Read some Parquet files in parallel.
rds = ray.data.read_parquet(s3_files)
print(type(rds))

2022-10-12 21:24:48,614	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


<class 'ray.data.dataset.Dataset'>


In [7]:
# Parquet stores the number of rows per file in the Parquet metadata, 
# so we can get the number of rows in rds without triggering a full data read!
print(f"Number rows: {rds.count()}")

# Parquet pulls size-in-bytes from its metadata (not triggering a data read)
# This could be significantly different than actual in-memory size!
print(f"Size bytes (from parquet metadata): {rds.size_bytes()}")
# Trigger full reading of the dataset and inspect the size in bytes.
print(f"Size bytes (from full data read): {rds.fully_executed().size_bytes()}")

# Fetch the schema from the underlying Parquet metadata.
print("\nSchema data types:")
data_types = list(zip(rds.schema().names, rds.schema().types))
[print(f"{s[0]}: {s[1]}") for s in data_types]

# Take a peek at a sample row
print("\nLook at a sample row:")
rds.take(1)

# Number rows: 6941024

Number rows: 6941024
Size bytes (from parquet metadata): 602373955


Read progress: 100%|█████████████████████████████| 1/1 [03:13<00:00, 193.05s/it]

Size bytes (from full data read): 573504109

Schema data types:
vendor_id: string
pickup_at: timestamp[us]
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
rate_code_id: string
store_and_fwd_flag: string
pickup_location_id: int32
dropoff_location_id: int32
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
improvement_surcharge: float
total_amount: float
congestion_surcharge: float

Look at a sample row:


[ArrowRow({'vendor_id': '1',
           'pickup_at': datetime.datetime(2019, 6, 1, 0, 55, 13),
           'dropoff_at': datetime.datetime(2019, 6, 1, 0, 56, 17),
           'passenger_count': 1,
           'trip_distance': 0.0,
           'rate_code_id': '1',
           'store_and_fwd_flag': 'N',
           'pickup_location_id': 145,
           'dropoff_location_id': 145,
           'payment_type': '2',
           'fare_amount': 3.0,
           'extra': 0.5,
           'mta_tax': 0.5,
           'tip_amount': 0.0,
           'tolls_amount': 0.0,
           'improvement_surcharge': 0.30000001192092896,
           'total_amount': 4.300000190734863,
           'congestion_surcharge': 0.0})]

<b>Filter on Read - Projection and Filter Pushdown</b>

Note that Ray Datasets' Parquet reader supports projection (column selection) and row filter pushdown, where we can push the above column selection and the row-based filter to the Parquet read. If we specify column selection at Parquet read time, the unselected columns won't even be read from disk!

The row-based filter is specified via [Arrow's dataset field expressions](https://arrow.apache.org/docs/6.0/python/generated/pyarrow.dataset.Expression.html#pyarrow.dataset.Expression). 

<b>Best practice is to filter as much as you can directly in the Ray Dataset read_parquet() statement! </b>

Normally there is some data exploration to determine the cleaning steps.  Let's just assume we know the data cleaning steps are:
- Drop negative trip distances, 0 fares, 0 passengers, less than 1min trip durations
- Drop 2 unknown zones ['264', '265']
- Calculate trip duration and add it as a new column


In [8]:
def pushdown_read_data(files_list: list,
                       sample_ids: list) -> Dataset:
    filter_expr = (
        (pds.field("passenger_count") > 0)
        & (pds.field("trip_distance") > 0)
        & (pds.field("fare_amount") > 0)
        & (~pds.field("pickup_location_id").isin([264, 265]))
        & (~pds.field("dropoff_location_id").isin([264, 265]))
        & (pds.field("dropoff_location_id").isin(sample_ids))
    )

    the_dataset = ray.data.read_parquet(
        files_list,
        columns=[
            'pickup_at', 'dropoff_at', 
            'pickup_location_id', 'dropoff_location_id',
            'passenger_count', 'trip_distance', 'fare_amount'], 
        filter=filter_expr,
    )

    # Force full execution of both of the file reads.
    the_dataset = the_dataset.fully_executed()
    return the_dataset

In [9]:
# Test the pushdown_read_data function
pushdown_ds = pushdown_read_data(s3_files, sample_locations)

print(f"Number rows: {pushdown_ds.count()}")
# Display some metadata about the dataset.
print("\nMetadata: ")
print(pushdown_ds)
# Fetch the schema from the underlying Parquet metadata.
print("\nSchema:")
print(pushdown_ds.schema())
# Take a peek at a single row
print("\nLook at a sample row:")
pushdown_ds.take(1)


2022-10-12 21:28:08,867	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Read progress: 100%|█████████████████████████████| 1/1 [01:48<00:00, 108.27s/it]

Number rows: 143759

Metadata: 
Dataset(num_blocks=1, num_rows=143759, schema={pickup_at: timestamp[us], dropoff_at: timestamp[us], pickup_location_id: int32, dropoff_location_id: int32, passenger_count: int8, trip_distance: float, fare_amount: float})

Schema:
pickup_at: timestamp[us]
dropoff_at: timestamp[us]
pickup_location_id: int32
dropoff_location_id: int32
passenger_count: int8
trip_distance: float
fare_amount: float
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2548

Look at a sample row:


[ArrowRow({'pickup_at': datetime.datetime(2019, 6, 1, 0, 29, 10),
           'dropoff_at': datetime.datetime(2019, 6, 1, 0, 33, 32),
           'pickup_location_id': 166,
           'dropoff_location_id': 238,
           'passenger_count': 1,
           'trip_distance': 1.0,
           'fare_amount': 5.5})]

In [10]:
# check sampling
df = pushdown_ds.to_pandas(limit=pushdown_ds.count())
print(df[["dropoff_location_id", "trip_distance"]].groupby("dropoff_location_id").count())


                     trip_distance
dropoff_location_id               
157                           2198
205                           1457
238                         140104


<b>Custom data transform functions</b>

Ray Datasets allows you to specify custom data transform functions using familiar syntax, such as Pandas.  These [<b>custom functions, or UDFs,</b>](https://docs.ray.io/en/latest/data/transforming-datasets.html) can be called using `rds.map_batches(my_UDF, batch_format="pandas")`.  It is necessary to specify the data processing API you are using in the `batch_format parameter`.

Available data processing APIs you can specify in the batch_format paramater include <b>"pandas", “pyarrow”, “numpy”</b>, and simple Python <b>list</b>.  Tabular data will be passed into your UDF by default as a pandas dataframe.  Tensor data will be passed into your UDF as a numpy array.

It is also possible to chain UDFs using [BatchMapper](https://docs.ray.io/en/latest/ray-air/check-ingest.html).  This is convenient when you want to chain a standard scaler step before training, for example.
<br>

In [11]:
# A Pandas DataFrame UDF for transforming the underlying blocks of a Dataset in parallel.
def transform_batch(the_df: pd.DataFrame) -> pd.DataFrame:
    df = the_df.copy()    
    df["trip_duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.seconds    
    df = df[df["trip_duration"] > 60]    
    df.drop(["dropoff_at", "pickup_at", "pickup_location_id"], axis=1, inplace=True)
    df['dropoff_location_id'] = df['dropoff_location_id'].fillna(-1)
    return df

In [12]:
# Test the transform UDF function
print(f"Before transform number rows: {pushdown_ds.count()}")

# batch_format="pandas" tells Datasets to provide the transformer with blocks
# represented as Pandas DataFrames.
pushdown_ds = pushdown_ds.map_batches(transform_batch, batch_format="pandas")

# verify row count
pushdown_rows = pushdown_ds.count()
print(f"After transform number rows: {pushdown_rows}")


Before transform number rows: 143759


Map_Batches: 100%|████████████████████████████████| 1/1 [00:00<00:00,  3.64it/s]

After transform number rows: 143277


In [13]:
# delete data to free up memory in our Ray cluster
del rds
del pushdown_ds

<b>Tidying up</b>

To make our code easier to read, let's summarize the data processing functions again here.

In [14]:
# Filter parquet data using Ray Datasets read_parquet()
def pushdown_read_data(files_list: list,
                       sample_ids: list) -> Dataset:
    
    start = time.time()
    
    filter_expr = (
        (pds.field("passenger_count") > 0)
        & (pds.field("trip_distance") > 0)
        & (pds.field("fare_amount") > 0)
        & (~pds.field("pickup_location_id").isin([264, 265]))
        & (~pds.field("dropoff_location_id").isin([264, 265]))
        & (pds.field("dropoff_location_id").isin(sample_ids))
    )

    the_dataset = ray.data.read_parquet(
        files_list,
        columns=[
            'pickup_at', 'dropoff_at', 
            'pickup_location_id', 'dropoff_location_id',
            'passenger_count', 'trip_distance', 'fare_amount'], 
        filter=filter_expr,
    )

    # Force full execution of both of the file reads.
    the_dataset = the_dataset.fully_executed()
    
    data_loading_time = time.time() - start
    print_time(f"Data loading time: {data_loading_time:.2f} seconds")
    return the_dataset

# A Pandas DataFrame UDF for transforming the underlying blocks of a Dataset in parallel.
def transform_batch(the_df: pd.DataFrame) -> pd.DataFrame:
    start = time.time()
    
    df = the_df.copy()    
    df["trip_duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.seconds    
    df = df[df["trip_duration"] > 60]    
    df.drop(["dropoff_at", "pickup_at", "pickup_location_id"], axis=1, inplace=True)
    df['dropoff_location_id'] = df['dropoff_location_id'].fillna(-1)
    
    data_transform_time = time.time() - start
    # print_time(f"Data transform time: {data_transform_time:.2f} seconds")
    return df

# Batch training with Ray Dataset <a class="anchor" id="train_func"></a>

Now that we have learned more about our data and written a pandas UDF to transform our data, we are ready to train a model on batches of this data in parallel.

To simplify the model training part, we will use linear regression in Scikit-learn.  
- We will use the `dropoff_location_id` column in the dataset to group the dataset into data batches. 
- Then we will fit a separate model for each batch to predict `trip_duration`.

In [15]:
# If necessary pip install scikit-learn from your terminal

import sklearn
from sklearn.base import BaseEstimator 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from ray.train.sklearn import SklearnTrainer, SklearnPredictor
from ray.train.batch_predictor import BatchPredictor

<b>Define training functions</b>

We want to fit a linear regression model to the trip duration for each drop-off location.  For scoring, we will calculate mean absolute error on the validation set, and report that as model error per drop-off location.

We define `fit_and_score_sklearn` function [<b><i>as a Ray task</i></b>](https://docs.ray.io/en/latest/ray-core/tasks.html), where each Scikit-learn training task will consume a dataset shard in batches.  Ray is able to automatically distribute ray tasks on your Ray cluster, to utilize parallel compute.


In [16]:
# Ray task to fit and score a scikit-learn model.
@ray.remote
def fit_and_score_sklearn(
    train_df: pd.DataFrame, test_df: pd.DataFrame, model: BaseEstimator
) -> Tuple[BaseEstimator, float]:
    
    # Assemble train/test pandas dfs
    train_X = train_df[["passenger_count", "trip_distance", "fare_amount"]]
    train_y = train_df.trip_duration
    test_X = test_df[["passenger_count", "trip_distance", "fare_amount"]]
    test_y = test_df.trip_duration
    
    # Start training.
    model = model.fit(train_X, train_y)
    pred_y = model.predict(test_X)
    error = sklearn.metrics.mean_absolute_error(test_y, pred_y)
    
    return str(model), error

<br>

The `train_and_evaluate` function contains the logic for train-test splitting and fitting of multiple models in parallel on each data batch, for purposes of comparison. Thanks to this, we can evaluate several models and choose the best one for each data batch.

This function takes as input, batches of Ray Dataset data.  Each batch of data is placed into Ray's distributed shared-memory object store, using the command [<b><i>ray.put()</i></b>](https://docs.ray.io/en/latest/ray-core/objects.html). Then the remote `fit_and_score_sklearn` Ray task is run simultaneously for all batches of data at once. Function `fit_and_score_sklearn` return values are all retrieved outside the loop using a single [<b><i>ray.get()</i></b>](https://docs.ray.io/en/latest/ray-core/tasks/patterns/ray-get-loop.html) call.
<br>


In [17]:
def train_and_evaluate(
    the_df: pd.DataFrame, 
    models: List[BaseEstimator]
) -> List[Tuple[BaseEstimator, float]]:
    
    # check if input df is big enough for training
    if len(the_df) < 4:
        print_time(f"Dataframe for LocID: {i} is empty or smaller than 4")
        return None
    else:
        loc_id = the_df.dropoff_location_id[0]
        # print(f"Processing location {loc_id}...")
    
    start = time.time()

    # Train / test split
    # Randomly split the data into 80/20 train/test.
    train_df, test_df = train_test_split(the_df, test_size=0.2, shuffle=True)
    
    # We put the train & test dataframes into Ray object store
    # so that they can be reused by all models fitted here.
    # https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html#tip-3-avoid-passing-same-object-repeatedly-to-remote-tasks
    train_ref = ray.put(train_df)
    test_ref = ray.put(test_df)

    # Launch a fit and score task for each model.
    results = ray.get(
        [fit_and_score_sklearn.remote(train_ref, test_ref, model) for model in models]
    )
    # results.sort(key=lambda x: x[1])  # sort by error
    
    # Assemble loc_id, name of model, and metrics in a pandas DataFrame
    results = [loc_id] + list(results[0])
    results_return = pd.DataFrame(columns=['location_id', 'model', 'error'])
    results_return.loc[0] = results

    training_time = time.time() - start
    print_time(f"Training time for LocID {loc_id}: {training_time:.2f} seconds")
    
    return results_return

<br>

Recall how we wrote a data transform <i>UDF_func</i> using Pandas syntax?  It was called with pattern:
- `rds.map_batches(UDF_func, batch_format)`

Similarly, a groupby-agg function can be used later when we perform a Ray Dataset <b>groupby</b>.  Below, the function <i>agg_func</i> will be called using a pattern: 
- `rds.groupby.map_groups(agg_func, batch_format)`.
<br>


In [18]:
# A Pandas DataFrame aggregation function for processing grouped batches of Ray Dataset data.
def agg_func(the_df: pd.DataFrame):
    
    ret = pd.DataFrame()
    
    # Handle errors in data groups
    try:
        # Transform the input pandas AND fit_and_evaluate the transformed pandas
        ret = train_and_evaluate(transform_batch(the_df), MODELS)
    except Exception:
        pass
    
    # Process null data groups
    if ret.shape[0] == 0:
        loc_id = the_df.dropoff_location_id[0]
        print(f"failed on {loc_id}")
        # assemble a null entry
        ret = [loc_id, None, None]
        results_return = pd.DataFrame(columns=['location_id', 'model', 'error'])
        results_return.loc[0] = ret
        return results_return
    
    # print_time(f"agg_func returned type: {type(ret)}")
    return ret
    

<b>Main driver code using Ray Datasets `map_groups`, on all the data!</b>

Finally, the main "driver code" reads each Parquet file (each file corresponds to one month of NYC taxi data) into a Ray Dataset, called `rds`. Then we use Ray Dataset <b>groupby</b> to map each group into a batch of data, on which `agg_func` can run, using the pattern [groupby-map_groups(agg_func, batch_format)](https://docs.ray.io/en/latest/data/api/grouped_dataset.html).  This implements an accumulator-based aggregation, which can run on each batch of data in parallel. 
<br>



In [22]:
# possible to add more trainers in this list
MODELS = [LinearRegression()]

In [26]:
# Driver code to run this.

PRINT_TIMES = False

SMOKE_TEST = False
if SMOKE_TEST:
    starting_idx = -1
    sample_locations = random.sample(list(local_utils.dataprep.location_ids), 3)
else:
    starting_idx = -3
    sample_locations = list(local_utils.dataprep.location_ids)

s3_files = [f"s3://{file}" for file in s3_partitions.files][starting_idx:]
print(f"NYC Taxi using {len(s3_files)} file(s)!")   
print_time(f"sample locations: {sample_locations}")

start = time.time()

# Read data into Ray Dataset
rds = pushdown_read_data(s3_files, sample_locations)

# Use Ray Dataset groupby.map_groups() to parallel process each group
# Returns a Ray Datset
results = rds.groupby("dropoff_location_id").map_groups(
            agg_func, batch_format="pandas")
print_time(f"groupby.map_groups() returned type: {type(results)}")

total_time_taken = time.time() - start
print(f"Total number of models: {len(sample_locations)}")
print(f"TOTAL TIME TAKEN: {total_time_taken:.2f} seconds")

NYC Taxi using 3 file(s)!


Map_Batches:  67%|████████████████████▋          | 2/3 [07:14<04:09, 249.38s/it]

(_map_block_nosplit pid=49034) failed on 75
(_map_block_nosplit pid=49034) failed on 79
(_map_block_nosplit pid=49034) failed on 154


Map_Batches: 100%|███████████████████████████████| 3/3 [07:46<00:00, 155.43s/it]

Total number of models: 259
TOTAL TIME TAKEN: 475.80 seconds


In [27]:
print(f"Total number of models: {len(sample_locations)}")
print(f"TOTAL TIME TAKEN: {total_time_taken:.2f} seconds")

# Note: Ran this on my M1 mac laptop with 8 CPU
# TODO: Run this on Anyscale bigger cluster

Total number of models: 259
TOTAL TIME TAKEN: 475.80 seconds


In [28]:
# Sort results ascending by error

print(type(results))

# sort values by ascending error
results_df = results.to_pandas(limit=results.count())
results_df.sort_values(by=["error"], ascending=True, inplace=True)
results_df

<class 'ray.data.dataset.Dataset'>


,location_id,model,error
102,105.0,LinearRegression(),48.927087
1,2.0,LinearRegression(),127.049780
248,253.0,LinearRegression(),179.224421
45,46.0,LinearRegression(),272.618932
240,245.0,LinearRegression(),295.203238
...,...,...,...
4,5.0,LinearRegression(),4285.400513
183,187.0,LinearRegression(),8575.909775
74,75.0,NaN,NaN
78,79.0,NaN,NaN
